In [54]:
!pip install pandas-profiling

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.9/390.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 82.4 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=6411409fd221281c8681cbbb39bc0f7f5cb20ad6de1ec672dadf4aaa8d04c97a
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin


In [55]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb
from ydata_profiling import ProfileReport

In [56]:
df = pd.read_csv('Rotten_Tomatoes_Movies3.csv')

In [57]:
profile = ProfileReport(df, title="Rotten Tomatoes Movies Report")
profile.to_file("rotten_tomatoes_movies_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
missing_values = df.isnull().sum()
print("Missing values in each column:")
print(missing_values)

Missing values in each column:
movie_title              0
movie_info              24
critics_consensus     8329
rating                   0
genre                   17
directors              114
writers               1349
cast                   284
in_theaters_date       815
on_streaming_date        2
runtime_in_minutes     155
studio_name            416
tomatometer_status       0
tomatometer_rating       0
tomatometer_count        0
audience_rating        252
dtype: int64


In [28]:
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

# Impute numerical columns with mean
imputer = SimpleImputer(strategy='mean')
df[numerical_cols] = imputer.fit_transform(df[numerical_cols])

# Impute categorical columns with the most frequent value
imputer_cat = SimpleImputer(strategy='most_frequent')
df[categorical_cols] = imputer_cat.fit_transform(df[categorical_cols])

In [29]:
genre_dummies = df['genre'].str.get_dummies(sep=',')
genre_dummies = genre_dummies.add_prefix('genre_')
df = pd.concat([df, genre_dummies], axis=1)

In [30]:
directors_dummies = df['directors'].str.get_dummies(sep=',')
directors_dummies = genre_dummies.add_prefix('directors_')
df = pd.concat([df, directors_dummies], axis=1)

In [31]:
rating_dummies = pd.get_dummies(df['rating'], prefix='rating_')
df = pd.concat([df, rating_dummies], axis=1)

In [32]:
tomatometer_status_dummies = pd.get_dummies(df['tomatometer_status'], prefix='tomatometer_status_')
df = pd.concat([df, tomatometer_status_dummies], axis=1)

In [33]:
df = df.applymap(lambda x: 1 if x is True else 0 if x is False else x)

<ipython-input-33-27d276b13a82>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: 1 if x is True else 0 if x is False else x)


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16638 entries, 0 to 16637
Columns: 101 entries, movie_title to tomatometer_status__Rotten
dtypes: float64(4), int64(85), object(12)
memory usage: 12.8+ MB


In [35]:
missing_values = df.isnull().sum()
print("Missing values in each column:")
print(missing_values)

Missing values in each column:
movie_title                            0
movie_info                             0
critics_consensus                      0
rating                                 0
genre                                  0
                                      ..
rating__R                              0
rating__R)                             0
tomatometer_status__Certified Fresh    0
tomatometer_status__Fresh              0
tomatometer_status__Rotten             0
Length: 101, dtype: int64


In [36]:
df.drop(columns=['genre'], inplace=True)
df.drop(columns=['directors'], inplace=True)
df.drop(columns=['rating'], inplace=True)
df.drop(columns=['tomatometer_status'], inplace=True)

In [40]:
# Convert 'in_theaters_date' to datetime format, specify dayfirst=True if necessary
df['in_theaters_date'] = pd.to_datetime(df['in_theaters_date'], errors='coerce', dayfirst=True)
df['on_streaming_date'] = pd.to_datetime(df['on_streaming_date'], errors='coerce', dayfirst=True)

# Handle missing values in 'in_theaters_date'
# Option 1: Impute with median year, month, and day (or other strategy)
median_year = df['in_theaters_date'].dropna().dt.year.median()
median_month = df['in_theaters_date'].dropna().dt.month.median()
median_day = df['in_theaters_date'].dropna().dt.day.median()

df['in_theaters_date'] = df['in_theaters_date'].fillna(
    pd.Timestamp(year=int(median_year), month=int(median_month), day=int(median_day))
)

# Extract year, month, and day after imputation
df['in_theaters_year'] = df['in_theaters_date'].dt.year
df['in_theaters_month'] = df['in_theaters_date'].dt.month
df['in_theaters_day'] = df['in_theaters_date'].dt.day

# Check if the issue is resolved
print("\nMissing values in 'in_theaters_date' related columns after fixing:")
print(df[['in_theaters_year', 'in_theaters_month', 'in_theaters_day']].isnull().sum())


Missing values in 'in_theaters_date' related columns after fixing:
in_theaters_year     0
in_theaters_month    0
in_theaters_day      0
dtype: int64


In [41]:
df_numeric = df.select_dtypes(include=['float64', 'int64'])
X = df_numeric.drop(columns=['audience_rating'])
y = df_numeric['audience_rating']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting splits
print("Training set features shape:", X_train.shape)
print("Testing set features shape:", X_test.shape)
print("Training set target shape:", y_train.shape)
print("Testing set target shape:", y_test.shape)


Training set features shape: (13310, 88)
Testing set features shape: (3328, 88)
Training set target shape: (13310,)
Testing set target shape: (3328,)


In [46]:
model = RandomForestRegressor(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse:.4f}")

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

# R-squared (R2)
r2 = r2_score(y_test, y_pred)
print(f"R-squared (R2): {r2:.4f}")

cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
print(f"Cross-Validation Mean Squared Error: {cv_scores.mean():.4f}")



Mean Absolute Error (MAE): 11.4548
Mean Squared Error (MSE): 210.0864
Root Mean Squared Error (RMSE): 14.4944
R-squared (R2): 0.4850
Cross-Validation Mean Squared Error: -209.8286


In [50]:
gb_model = GradientBoostingRegressor(random_state=42)
xgb_model = XGBRegressor(random_state=42)
lgb_model = lgb.LGBMRegressor(random_state=42)

models = {'Gradient Boosting': gb_model, 'XGBoost': xgb_model, 'LightGBM': lgb_model}

for model_name, model in models.items():
    print(f"\nTraining and evaluating {model_name}...")

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print(f"{model_name} Performance:")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"R-squared (R2): {r2:.4f}")



Training and evaluating Gradient Boosting...
Gradient Boosting Performance:
Mean Absolute Error (MAE): 11.1107
Mean Squared Error (MSE): 195.6391
Root Mean Squared Error (RMSE): 13.9871
R-squared (R2): 0.5204

Training and evaluating XGBoost...
XGBoost Performance:
Mean Absolute Error (MAE): 11.3343
Mean Squared Error (MSE): 206.9452
Root Mean Squared Error (RMSE): 14.3856
R-squared (R2): 0.4927

Training and evaluating LightGBM...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005736 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 651
[LightGBM] [Info] Number of data points in the train set: 13310, number of used features: 72
[LightGBM] [Info] Start training from score 60.452719
LightGBM Performance:
Mean Absolute Error (MAE): 11.0131
Mean Squared Error (

In [53]:
param_dist = {
    'num_leaves': np.arange(20, 150, 10),  # Number of leaves in each tree
    'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],  # Learning rate
    'n_estimators': [100, 200, 500],  # Number of trees
    'max_depth': [3, 5, 7, 10, -1],  # Maximum depth of each tree (-1 means no limit)
    'min_data_in_leaf': [20, 40, 60],  # Minimum data in leaf node
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],  # Fraction of samples used for fitting
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],  # Fraction of features for each tree
    'reg_alpha': [0, 0.1, 0.5, 1],  # L1 regularization term
    'reg_lambda': [0, 0.1, 0.5, 1],  # L2 regularization term
}

lgb_model = lgb.LGBMRegressor(random_state=42)

random_search = RandomizedSearchCV(lgb_model, param_dist, n_iter=50, scoring='neg_mean_squared_error', cv=5, verbose=2, random_state=42, n_jobs=-1)

random_search.fit(X_train, y_train)

print("Best hyperparameters:", random_search.best_params_)

y_pred_best = random_search.best_estimator_.predict(X_test)

mae_best = mean_absolute_error(y_test, y_pred_best)
mse_best = mean_squared_error(y_test, y_pred_best)
rmse_best = np.sqrt(mse_best)
r2_best = r2_score(y_test, y_pred_best)

print(f"Optimized LightGBM Performance:")
print(f"Mean Absolute Error (MAE): {mae_best:.4f}")
print(f"Mean Squared Error (MSE): {mse_best:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_best:.4f}")
print(f"R-squared (R2): {r2_best:.4f}")


Fitting 5 folds for each of 50 candidates, totalling 250 fits


/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The LGBMRegressor or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005166 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 651
[LightGBM] [Info] Number of data points in the train set: 13310, number of used features: 72
[LightGBM] [Info] Start training from score 60.452719
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p